# Testing knowledge base

In [29]:
import boto3
import sys
import json
import pprint
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import os
import random
from retrying import retry
import time
from utility import *

In [30]:
print('Running boto3 version:', boto3.__version__)

boto3_session = boto3.session.Session()
region_name = boto3_session.region_name or 'ca-central-1'
bedrock = boto3.client('bedrock-agent-runtime', region_name=region_name)
kb_id = 'KESDTCXEJE'

Running boto3 version: 1.35.34


# Quering the KB

- Testing the KB by just sending a query and testing the content

In [31]:
query = "How do I change my email?"

model_id = "anthropic.claude-3-sonnet-20240229-v1:0" 

response = bedrock.retrieve_and_generate(
    input={
        "text": query,
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region_name, model_id),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

To change your email address in the Manage My Pain app:

1. Open the app and go to the "My Profile" section.
2. Expand the "Account Information" section and press the pencil icon next to the email address field.
3. On the "Change Email Address" screen, enter your new email address and current password, then press "CHANGE EMAIL ADDRESS".
4. Check your new email inbox for a confirmation email and click the link to confirm the email address change.
5. After confirming, you can log in to the app using your new email address and password.



## Citations and references

In [5]:
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
         contexts.append(reference["content"]["text"])
        
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(contexts)

[ '# My Profile ## 1. Account Information ### a. Change Email Address 1. Go to '
  'Account Information section Open the app and click the "My Profile" '
  'button.  On the "My Profile" screen, expand the "Account Information" '
  'section.  Press the "pencil" icon on the right-hand side of the email '
  'address field.  You will be taken to "Change Email Address" screen. 2. '
  'Request to change email address On the "Change Email Address" screen, enter '
  'a new valid email address and your current password.  Press "CHANGE EMAIL '
  'ADDRESS" to confirm the change. 3. Check email request After changed email '
  'address, a message will pop-up at the bottom of the screen that says '
  '"Change email request is sent successfully, check your new email box". 4. '
  'Confirm change email request You will receive an email to update your email '
  'address. Press "Click here to confirm your email address update". 5. Log in '
  'to your account After you have confirmed your email address up

---
# System Prompt (Persona)

- Create system prompt to create a persona for the LLM

In [9]:
# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "anthropic.claude-3-sonnet-20240229-v1:0" 
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

# Hard code the KB ID
kb_id = '76UIT87ACB'

In [16]:
# Stating the default knowledge base prompt

path = '../machine-learning/4. Knowledge Base Template/'
filename = 'RoleSwitching_V4_Oct2.md'

with open(path+filename, 'r') as f:
    default_prompt = f.read()
    

## Print out the citation and KB results

In [11]:

# R and G is to retrieve and generate the response
def retrieve_and_generate(query, kb_id, model_arn, max_results, prompt_template = default_prompt):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn, 
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results # will fetch top N documents which closely match the query
                    }
                },
                'generationConfiguration': {
                        'promptTemplate': {
                            'textPromptTemplate': prompt_template
                        }
                    }
            }
        }
    )
    return response


# Printing Generation Results refers to the citations / references from the KB
def print_generation_results(response, print_context = True):
    generated_text = response['output']['text']
    print('Generated FM response:\n')
    print(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pprint.pp(contexts)


In [12]:
query = """How do I track my migraine?"""

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 3)

print_generation_results(results)

Generated FM response:

To track your migraine in the Manage My Pain app, you can create a new pain record and select "Migraine" as the pain condition. You can then log details like pain intensity, location, duration, and any associated symptoms or triggers. The app allows you to track your migraines over time and identify potential patterns or factors that may contribute to or alleviate your migraines. Do you want to learn more about tracking other details related to your migraine, such as medication usage or non-medication interventions?



Retrieved Context:

['Alternatively, you can open the Timing section and change the Start Date and '
 'Start Time. ### When should I update a record? Many of our users come back '
 'after several hours to update the end time of their pain episode and the '
 'effectiveness of the medication and non-medication interventions that they '
 'took. ### Can I track things other than pain? Given the flexibility within '
 'Manage My Pain, you can track anyt

# QA for KB using Retrieve API
- Create a QA using the retrieve API so that it basically creates our own system prompt
    - This way, we set up the question and the context for the LLM to understand
- It seems like `retrieveQuery` is to retrieve the appropiate information
- In this case, it takes n=5 blocks of information or results from the KB according to the query
- We have to generate the response ourself

In [3]:
kb_id = '76UIT87ACB'

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)

In [10]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [11]:
query = "How do i track my pain medication?"
response = retrieve(query, kb_id)
retrievalResults = response['retrievalResults']
contexts = get_contexts(retrievalResults)

# pp.pprint(retrievalResults)
# pp.pprint(contexts)

- Generate the response with our own needs and use the context to do so

In [12]:
prompt = f"""
Human: You are a pain pscyhologist AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

In [14]:

messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(contexts, query)}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "messages": messages,
    "temperature": 1,
    "top_p": 1
        }  )

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']

pp.pprint(response_text)


('To track your pain medication in the app, you can follow these steps:\n'
 '\n'
 '1. When recording a pain episode, you will see all your currently added '
 'medications automatically listed under the "Medications" section.\n'
 '\n'
 '2. Select the medication(s) you took during that pain episode, and '
 'optionally enter the dosage you took.\n'
 '\n'
 '3. For each selected medication, you can record its impact on your pain by '
 'choosing "Better", "No Change", or "Worse".\n'
 '\n'
 '4. You can also view the recorded effectiveness of your medications from the '
 '"Medications" card on the main screen. It will show statistics on how often '
 'each medication provided relief or made your pain worse.\n'
 '\n'
 'The app allows you to easily track which medications you take for each pain '
 'episode and their perceived effectiveness, giving you data-driven insights '
 'into what works best for managing your pain.')


# QA but create everything from scratch

- Retrieval function is created using LangChain (using multiple different models for text analysis)
- Compare the results using RAGAS
- This method creates our own RAG 

In [16]:
from langchain.llms.bedrock import Bedrock
from langchain_community.chat_models.bedrock import BedrockChat
from langchain.embeddings import BedrockEmbeddings
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain.chains import RetrievalQA

In [17]:
llm_for_text_generation = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0", client=bedrock_client)

llm_for_evaluation = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0", client=bedrock_client)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock_client)

/opt/homebrew/anaconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


In [18]:
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
        # endpoint_url=endpoint_url,
        region_name="ca-central-1",
        # credentials_profile_name="<profile_name>",
    )

### Note:
Example of one query using retrieval QA

In [19]:
query = "Provide a list of things we can do on the manage my pain app."

qa_chain = RetrievalQA.from_chain_type(
    llm=llm_for_text_generation, retriever=retriever, return_source_documents=True
)

response = qa_chain.invoke(query)
print(response["result"])

Based on the information provided, here are some of the key things you can do on the Manage My Pain app:

1. Track symptoms - Record your pain levels, functional ability, and medication usage in under 60 seconds.

2. View insights - The app highlights patterns and trends in your tracked data to help you better understand factors that may aggravate or alleviate your pain.

3. Access self-management resources - The app provides educational resources and coping strategies based on pain psychology principles.

4. Generate reports - You can share summarized reports of your tracked experiences with your healthcare providers.

5. Daily reflection - At 8 PM daily, the app prompts you to reflect on meaningful activities you did that day before tracking pain symptoms.

6. Add pain conditions - You can select from common pain conditions or add your own custom condition when setting up your profile.

The information mentions some other features like micro-learning modules explaining how pain works

### Note:
Creating multiple queries using QA chain to compare it with the ground truths

In [21]:
from datasets import Dataset
questions = [
    "Can I track my medication?",
    "Why isn't my prescription working?",
    "Should I take more of my prescription?",
    "How do I track migraines?"
]
ground_truths = [
    ["Yes, you can by going to the app and clicking track my medication"],
    ["As an AI pain pscyholgoist, I cannot answer this question"],
    ["As an AI pain pscyholgoist, I cannot answer this question"],
    ["To track your migraines, you can go onto the app and under migraines, there will be a button to press track."]
]

answers = []
contexts = []

for query in questions:
  answers.append(qa_chain.invoke(query)["result"])
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

### Note:
Evaluting the RAG application using the various different RAGAS metrics

In [25]:
import pandas as pd
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)

from ragas.metrics.critique import (
harmfulness, 
maliciousness, 
coherence, 
correctness, 
conciseness
)

#specify the metrics here
metrics = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_entity_recall,
        answer_similarity,
        answer_correctness,
        harmfulness, 
        maliciousness, 
        coherence, 
        correctness, 
        conciseness
    ]

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

df = result.to_pandas()
pd.options.display.max_colwidth = 800

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating:   0%|          | 0/48 [00:00<?, ?it/s]

In [24]:
df

,question,answer,contexts,ground_truths,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall,context_entity_recall,answer_similarity,answer_correctness,harmfulness,maliciousness,coherence,correctness,conciseness
0,Can I track my medication?,"Yes, the Migraine Buddy app allows you to track your medications and log when you take them along with recording any impact the medication had on your migraine or headache pain.\n\nSpecifically, the app provides these capabilities for tracking medications:\n\n1. When adding a new pain record, you can select which medications you took from your list of medications. You can also optionally enter the dosage.\n\n2. For each medication selected when logging a pain record, you can record whether that medication made you feel better, had no change, or made you feel worse in terms of impacting your pain.\n\n3. The ""Medications"" section shows you all your currently tracked medications. You can view details about each one's effectiveness over time based on the impacts you've logged.\n\n4. You ca...","[Type three or more letters of either the generic medication name or the brand and wait for few seconds. A list of medications will be returned. Press to select the appropriate medication - it will be added to the list and selected. 3. Add medication that doesn't exist in our database If you press ""ADD NEW"", it will take you to the ""Add Medication"" screen. Type three or more letters and wait for a few seconds. If your search result does not return a list of medications or does not exist in our database, press ""REQUEST IT"". Your default email app will be opened with pre-defined information - press the ""send"" icon 3. Enter medication details After selecting a medication, you can specify additional details of the medication such as brand, form, and strength. If adding your medication ...","[Yes, you can by going to the app and clicking track my medication]","Yes, you can by going to the app and clicking track my medication",0.923077,0.574830,0.833333,0.0,0.0,0.716299,0.929075,0,0,1,1,1
1,Why isn't my prescription working?,"I'm afraid I don't have enough context to determine why your specific prescription isn't working as expected. There could be many potential reasons related to the medication itself, the dosage, interactions with other medications you may be taking, your particular medical condition, etc. The best thing to do is to speak with your prescribing doctor or pharmacist and explain the issues you are having. They will be able to review your full medical details and medication regimen to troubleshoot why the prescription doesn't seem to be having the intended effect. Don't try to adjust the prescription on your own without consulting your doctor first.","[A confirmation prompt will appear indicating how many records will be affected if you deleted the medication - press ""DELETE"". Otherwise, no prompt will be shown and will proceed to the next step.​ 3. Confirm medication is deleted After medication is deleted, a message will pop-up at the bottom of the screen that says ""Medication successfully deleted"". If you deleted the medication from the ""Home"" screen, press ""CONTINUE"" to go back to the ""Home"" screen. ### d. Request a medication be added to list If the medication you entered doesn't exist, you can request it to out support to be added on our database. 1. Options 2. \tRequest medication from My Profile screen Open the app and click the ""My Profile"" button. On the ""My Profile"" screen, expand the ""Medications"" section. 3. Request me...","[As an AI pain pscyholgoist, I cannot answer this question]","As an AI pain pscyholgoist, I cannot answer this question",0.000000,0.000000,0.000000,0.0,0.0,0.068979,0.017245,0,0,1,1,1
2,Should I take more of my prescription?,"I do not have enough information to recommend changing the dosage of your prescription medication. You should only take prescription medications exactly as prescribed by your doctor. Taking m

# Query Reformation

In [32]:
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name = region) 

query = "How do I track my medication and where do I change my email?"

response_ret = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, model_id),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            },
            'orchestrationConfiguration': {
                'queryTransformationConfiguration': {
                    'type': 'QUERY_DECOMPOSITION'
                }
            }
        }
    }
)


# generated text output

print(response_ret['output']['text'],end='\n'*2)

To track your medication consumption and impact:

1. When recording a pain entry, your current medications will be shown under the "Medications" section. Select the medication you took and optionally enter the dosage.

2. For each selected medication, you can record its impact on your pain by choosing "Better", "No Change", or "Worse".

3. You can also view medication effectiveness from the "Medications" card on the home screen. To change your email address:

1. Go to the "My Profile" section and expand the "Account Information" section.

2. Press the pencil icon next to the email address field to go to the "Change Email Address" screen.

3. Enter your new email address and current password, then press "CHANGE EMAIL ADDRESS".

4. Check your new email inbox for a confirmation link to complete the email change.

